<a href="https://colab.research.google.com/github/amirhbv/UT-AI-Spring99-CAs/blob/master/CA1/CA1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AI - Computer Assignment 1

## Search

In this assignment we are going to find answer for a problem using different search algorithms. As we know there are uninformed and informed algorithms. here we use 2 uninformed search algorithms, BFS and IDS and A* as informed saerch algorithm.

### Modeling
First we have to model the problem to know different states of it. Every state of the problem consists of the position fo the ambulance, the positions of the patients, the positions of the hospitals and their capacity and, the positions of the obstacles. We store every state of the problem in an instance of Map class. The Map class consists of a tuple for the ambulance position, a dictionary with hospital position as key and hospital capacity as value, and 2 lists for patiens and obstacles positions.

Start state of the problem is the equivalent instance of Map class to the given file as input. The start state can be instantiated using buildMapFromFile classmethdo of Map class.

We have reached the goal state of the problem when there are no patients in the map anymore, i.e. every patient is in a hospital.

The only moving objects in the map are the ambulance and the patients. The patients can only be moved by the ambulance. The ambulance can go in 4 directions (up, down, right, left). The ambulance can not go over the obstacles.
If there is patient in the new position of the ambulance, the patient must go in the same direction as the ambulance but the patient can not go over the obstacles or other patients, therfore the ambulance can not go in that direction. The patients will be removed from the map when arrived at a hospital with a non-zero capacity.

Cost of the answer found for the problem is the number of moves made by the ambulance to move every patient to the hospitals.

The following block is definition of the Map class.


In [0]:
import operator

def moveTuple(pos: tuple, d: tuple) -> tuple:
    return tuple(map(operator.add, pos, d))

def manhattanDistance(a: tuple, b: tuple) -> int:
    x1, y1 = a
    x2, y2 = b
    return abs(x2 - x1) + abs(y2 - y1)

def squaredEuclideanDistance(a: tuple, b: tuple) -> int:
    dx, dy = tuple(map(operator.sub, a, b))
    return dx * dx + dy * dy

class Map:
    def __init__(self, ambulance: tuple, hospitals: dict, patients: list, obstacles: list):
        self.ambulance = ambulance
        self.hospitals = dict(hospitals)
        self.obstacles = list(obstacles)
        self.patients = list(patients)

    def canMovePatient(self, pos: tuple, d: tuple) -> bool:
        newPos = moveTuple(pos, d)
        return newPos not in self.obstacles and newPos not in self.patients

    def canMoveAmbulance(self, d: tuple) -> bool:
        newAmbulancePos = moveTuple(self.ambulance, d)
        return newAmbulancePos not in self.obstacles and \
            (newAmbulancePos not in self.patients or self.canMovePatient(newAmbulancePos, d))

    def move(self, d: tuple) -> bool:
        """
        Move the ambulance if d is a valid move
        and returns True, returns False otherwise
        """
        if not self.canMoveAmbulance(d):
            return False

        self.ambulance = moveTuple(self.ambulance, d)
        try:
            patientIndex = self.patients.index(self.ambulance) # raises ValueError
            newPatientPos = moveTuple(self.patients[patientIndex], d)

            if newPatientPos in self.hospitals and self.hospitals[newPatientPos]:
                self.hospitals[newPatientPos] -= 1
                self.patients.pop(patientIndex)
            else:
                self.patients[patientIndex] = newPatientPos

        except ValueError:
            pass

        return True

    def findNearestHospitalDistance(self, patient: tuple, distanceFunction) -> int:
        minDist = -1
        for hospital in self.hospitals:
            dist = distanceFunction(patient, hospital)
            minDist = dist if dist < minDist or minDist == -1 else minDist

        return minDist

    def findPatientsToNearestHospitalDistance(self, distanceFunction) -> int:
        res = 0
        for patient in self.patients:
            res += self.findNearestHospitalDistance(patient, distanceFunction)

        return res

    def findAmbulanceToNearestHospitalDistance(self, distanceFunction) -> int:
        res = 0
        for patient in self.patients:
            res += (distanceFunction(patient, self.ambulance) + self.findNearestHospitalDistance(patient, distanceFunction))

        return res

    @property
    def isGoal(self) -> bool:
        return len(self.patients) == 0

    @classmethod
    def buildMapFromMap(cls, map):
        return cls(
            ambulance=map.ambulance,
            hospitals=map.hospitals,
            patients=map.patients,
            obstacles=map.obstacles
        )

    @classmethod
    def buildMapFromFile(cls, inputFileName: str):
        ambulance = ()
        hospitals = {}
        obstacles = []
        patients = []

        with open(inputFileName, 'r') as fin:
            for row, line in enumerate(fin):
                for col, cell in enumerate(line):
                    curr = (row, col)

                    if cell == 'A':
                        ambulance = curr
                    elif cell.isdigit():
                        hospitals[curr] = int(cell)
                    elif cell == '#':
                        obstacles.append(curr)
                    elif cell == 'P':
                        patients.append(curr)

        return cls(
            ambulance=ambulance,
            hospitals=hospitals,
            patients=patients,
            obstacles=obstacles
        )

    def __str__(self):
        return str((
            self.ambulance,
            tuple(self.hospitals.items()),
            tuple(self.patients)
        ))

    def __lt__(self, other):
        return len(self.patients) < len(other.patients)

    def __eq__(self, other):
        return hash(self) == hash(other)

    def __hash__(self):
        return hash((
            self.ambulance,
            tuple(self.hospitals.items()),
            tuple(self.patients)
        ))


Different methods are provided in class to use in different cases, hash, lt and eq are used when instances of the class are stored in a set o a priority queue. By using `buildMapFromMap` and `buildMapFromFile` methods we can copy a state or get the start state from a file. The other methods handle details explained above.

### Implementing Algorithms
We are going to implement 3 different algorithms. We Implement the algorithms as diferent methods of a class. The class also has a `ResultGenerator` to make the result format always the same in different algorithms. The implementation of the class is provided in the next block.

In [0]:
from queue import LifoQueue, PriorityQueue, Queue
from typing import List


class SearchProblem:
    class ResultGenerator:
        KEY_DEPTH = "Depth"
        KET_SUCCEEDED = "Succeeded"
        KEY_TOTAL_STATES = "Total States"
        KEY_UNIQUE_STATES = "Unique States"

        @classmethod
        def failure(cls) -> dict:
            return {
                cls.KET_SUCCEEDED: False,
                cls.KEY_DEPTH: -1,
                cls.KEY_TOTAL_STATES: -1,
                cls.KEY_UNIQUE_STATES: -1,
            }

        @classmethod
        def success(cls, depth=0, totalStatesCount=0, uniqueStatesCount=0) -> dict:
            return {
                cls.KET_SUCCEEDED: True,
                cls.KEY_DEPTH: depth,
                cls.KEY_TOTAL_STATES: totalStatesCount,
                cls.KEY_UNIQUE_STATES: uniqueStatesCount,
            }

    def __init__(self, inputFileName: str):
        self.inputFileName = inputFileName
        self.POSSIBLE_MOVES = [
            (1, 0),
            (0, 1),
            (-1, 0),
            (0, -1),
        ]

    def getStartState(self) -> Map:
        return Map.buildMapFromFile(self.inputFileName)

    def getSuccessors(self, state: Map) -> List[Map]:
        res: List[Map] = []

        for d in self.POSSIBLE_MOVES:
            newState = Map.buildMapFromMap(state)
            valid = newState.move(d)
            if valid:
                res.append(newState)

        return res

    def bfs(self):
        startState: Map = self.getStartState()
        if startState.isGoal:
            return self.ResultGenerator.success()

        frontier = Queue()
        frontier.put((startState, 0))

        visited = set()

        totalStatesCount = 0
        while not frontier.empty():
            totalStatesCount += 1

            currentState, depth = frontier.get()
            visited.add(currentState)

            depth += 1
            for state in self.getSuccessors(currentState):
                if state not in visited:
                    if state.isGoal:
                        return self.ResultGenerator.success(
                            depth=depth,
                            totalStatesCount=totalStatesCount,
                            uniqueStatesCount=len(visited)
                        )

                    frontier.put((state, depth))

        return self.ResultGenerator.failure()

    def dls(self, startState: Map, maxDepth: int):
        frontier = LifoQueue()
        frontier.put((startState, 0))

        visited = set()
        visitDepth = {}

        totalStatesCount = 0
        while not frontier.empty():
            totalStatesCount += 1

            currentState, depth = frontier.get()
            visited.add(currentState)
            visitDepth[currentState] = depth


            if depth == maxDepth:
                continue

            depth += 1
            for state in self.getSuccessors(currentState):
                if state not in visited or visitDepth[state] > depth:
                    if state.isGoal:
                        return self.ResultGenerator.success(
                            depth=depth,
                            totalStatesCount=totalStatesCount,
                            uniqueStatesCount=len(visited)
                        )

                    frontier.put((state, depth))

        return self.ResultGenerator.failure()


    def ids(self):
        startState: Map = self.getStartState()
        if startState.isGoal:
            return self.ResultGenerator.success()

        maxDepth = 1
        res = self.ResultGenerator.failure()
        while not res[self.ResultGenerator.KET_SUCCEEDED]:
            res = self.dls(startState, maxDepth)
            maxDepth += 1

        return res

    def astar(self, heuristic):
        startState: Map = self.getStartState()
        if startState.isGoal:
            return self.ResultGenerator.success()

        frontier = PriorityQueue()
        frontier.put((heuristic(startState), (startState, 0)))

        visited = set()

        totalStatesCount = 0
        while not frontier.empty():
            totalStatesCount += 1

            _, stateDepth = frontier.get()
            currentState, depth = stateDepth
            visited.add(currentState)

            depth += 1
            for state in self.getSuccessors(currentState):
                if state not in visited:
                    if state.isGoal:
                        return self.ResultGenerator.success(
                            depth=depth,
                            totalStatesCount=totalStatesCount,
                            uniqueStatesCount=len(visited)
                        )

                    frontier.put((heuristic(state) + depth, (state, depth)))

        return self.ResultGenerator.failure()


The `getStartState` method always gives a map instance built from the given input file and the `getSuccessors` methods gives a list of map instances which can be reached from the given state and are valid moves. There three methods for `bfs`, `ids` and `astar`. In all algorithms we have a frontier set and a visited set. Now we are going to explain each algorithm.

#### BFS
The `BFS` algorithm is an uninformed search algorithm. We use a Queue (FIFO) as the frontier set and we expand the shallowest states first. First we check if the start state is goal or not, then in each turn we visit the current state which is the shallowest in the frontier set and after that we check its neighbours which are not visited yet. if one of the neighbours is goal state the search is finished and the result is generated otherwise the neighbour is added to the frontier set. This algorithm is complete and is also optimal. It has a time complexity and space complexity of $O(b^d)$ where $b$ is the maximum branching factor of the search tree and $d$ is the maximum depth of the optimal solution.

#### IDS
The `IDS` algorithm or `Iterative Deepening Search` is another uninformed search algorithm. It uses depth limited `DFS` algorithm with different max depth. Let's first explain `DFS`,
> #### DFS
It is another uninformed search algorithm. A Stack (LIFO) is used as the frontier set and the deepest states are expanded first, i.e. in a branch we go as deep as possible and then we traverse back up. It may be slower than `BFS` and will just find the first answer which may not be an optimal solution to the problem. It also fails in infinite-depth spaces. The time complexity of `DFS` is $O(b^m)$ where $b$ is the maximum branching factor of the search tree and $m$ is the maximum depth of the search tree, which is terrible if $m$ is much larger than $d$ but the space complexity of this algorithm is $O(bm)$ which is linear and much better than space complexity of `BFS`.

In `IDS` we try to have space complexity of `DFS` and optimality of `BFS`. First we run `DFS` on state space and check if an answer is found, if no answer was found then we go with max depth of 2 and so on. As we use `DFS` so we have linear space complexity of $O(bd)$ and time complexity of $O(b^d)$. It is complete and is optimal when out step cost is the same for all actions.

#### A*
The `A*` algorithm is an informed search algorithm. the `A*` algorithm uses an evaluation funciton to choose the next state to expand. The evaluation function consists of 2 parts: $f(x) = g(x) + h(x)$, $g(x)$ is the actual cost of reaching state x from the start state (cost so far) and $h(x)$ is our heuristic function which the estimated cost of reaching goal state from state x. Here a PriorityQueue is used as the frontier set we will expand the state with lowest evaluation value first. The `A*` is complete and is optimal if the heuristic is admissible, a heuristic is admissible if for every state $n$ the $h(n)$ is lower than or equal to actual cost of reaching goal from state n.

Two differenet heuristics are used as the heuristic function:
1. Sum of Distance of Each Patient to Its Nearest Hospital (h1)
2. Sum of Distance of Ambulance to Each Patient and Then to Its Nearest Hospital (h2)

In both algorithms Manhattan distance is used as the distance function. The manhattan distance between 2 points $(x1, y1)$ and $(x2, y2)$ is $|x1 - x2| + |y1 - y2|$.

The first heuristic is admissible as each patient must at least go this number of moves the reach a hospital. But second one may not be admissible.


Now lets run these algorithms on testcases:


First we define heuristics to use in `A*`

In [0]:
def h1(state: Map):
    return state.findPatientsToNearestHospitalDistance(manhattanDistance)
    # return state.findPatientsToNearestHospitalDistance(squaredEuclideanDistance)

def h2(state: Map):
    return state.findAmbulanceToNearestHospitalDistance(manhattanDistance)


And then a utility function to run algorithms on a given problem:


In [0]:

from time import time
from prettytable import PrettyTable

def test(problem: SearchProblem, repeatCount=3):
    table = PrettyTable()
    table.field_names = [
        'Algorithm',
        SearchProblem.ResultGenerator.KEY_DEPTH,
        SearchProblem.ResultGenerator.KEY_TOTAL_STATES,
        SearchProblem.ResultGenerator.KEY_UNIQUE_STATES,
        'Average Time'
    ]

    totalTime = 0
    for i in range(repeatCount):
        start = time()
        res = problem.bfs()
        totalTime += (time() - start)

    table.add_row([
        'BFS',
        res[SearchProblem.ResultGenerator.KEY_DEPTH],
        res[SearchProblem.ResultGenerator.KEY_TOTAL_STATES],
        res[SearchProblem.ResultGenerator.KEY_UNIQUE_STATES],
        totalTime / repeatCount
    ])

    totalTime = 0
    for i in range(repeatCount):
        start = time()
        res = problem.ids()
        totalTime += (time() - start)

    table.add_row([
        'IDS',
        res[SearchProblem.ResultGenerator.KEY_DEPTH],
        res[SearchProblem.ResultGenerator.KEY_TOTAL_STATES],
        res[SearchProblem.ResultGenerator.KEY_UNIQUE_STATES],
        totalTime / repeatCount
    ])

    totalTime = 0
    for i in range(repeatCount):
        start = time()
        res = problem.astar(h1)
        totalTime += (time() - start)

    table.add_row([
        'A* (h1)',
        res[SearchProblem.ResultGenerator.KEY_DEPTH],
        res[SearchProblem.ResultGenerator.KEY_TOTAL_STATES],
        res[SearchProblem.ResultGenerator.KEY_UNIQUE_STATES],
        totalTime / repeatCount
    ])

    totalTime = 0
    for i in range(repeatCount):
        start = time()
        res = problem.astar(h2)
        totalTime += (time() - start)

    table.add_row([
        'A* (h2)',
        res[SearchProblem.ResultGenerator.KEY_DEPTH],
        res[SearchProblem.ResultGenerator.KEY_TOTAL_STATES],
        res[SearchProblem.ResultGenerator.KEY_UNIQUE_STATES],
        totalTime / repeatCount
    ])

    print(table)


Now we run tests for each input file:


In [0]:
test(SearchProblem('./1.in'))

+-----------+-------+--------------+---------------+-----------------------+
| Algorithm | Depth | Total States | Unique States |      Average Time     |
+-----------+-------+--------------+---------------+-----------------------+
|    BFS    |   11  |     441      |      273      |  0.020214557647705078 |
|    IDS    |   11  |     709      |      465      |  0.06092262268066406  |
|  A* (h1)  |   11  |     133      |      109      |  0.006929874420166016 |
|  A* (h2)  |   11  |      40      |       38      | 0.0020230611165364585 |
+-----------+-------+--------------+---------------+-----------------------+


In [0]:
test(SearchProblem('./2.in'))

+-----------+-------+--------------+---------------+--------------------+
| Algorithm | Depth | Total States | Unique States |    Average Time    |
+-----------+-------+--------------+---------------+--------------------+
|    BFS    |   27  |    34492     |     13883     | 1.2941004435221355 |
|    IDS    |   27  |    46960     |     16176     | 10.476608912150065 |
|  A* (h1)  |   27  |    12628     |      7654     | 1.0422022342681885 |
|  A* (h2)  |   31  |     6270     |      4441     | 0.4840404987335205 |
+-----------+-------+--------------+---------------+--------------------+


In [0]:
test(SearchProblem('./3.in'))

+-----------+-------+--------------+---------------+--------------------+
| Algorithm | Depth | Total States | Unique States |    Average Time    |
+-----------+-------+--------------+---------------+--------------------+
|    BFS    |   39  |    253722    |     28708     | 10.265942255655924 |
|    IDS    |   39  |    100921    |     24938     | 38.18173805872599  |
|  A* (h1)  |   39  |    24652     |      9823     | 1.9341645240783691 |
|  A* (h2)  |   39  |     2210     |      1554     | 0.1358013947804769 |
+-----------+-------+--------------+---------------+--------------------+
